## 핸즈온 머신러닝 7장 연습문제

### 1. 정확히 같은 훈련 데이터로 다섯 개의 다른 모델을 훈련시켜서 모두 95% 정확도를 얻었다면 이 모델들을 연결하여 더 좋은 결과를 얻을 수있을까요? 가능하다면 어떻게 해야 할까요? 그렇지 않다면 왜일까요?


좋은 결과가 나올 것이다. 각 분류기가 약한 학습기여도 충분하게 많고 다양하다면 앙상블은 높은 정확도를 가진 강한 학습기가 될 수 있기 때문이다.
모델이 서로 독립적이거나, 다양하거나, 다른 훈련 샘플에서 훈련되었다면 앙상블은 더더욱 좋다. 그렇지 않더라도 모델이 서로 다르면 좋다.

### 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?

직접 투표는 여러 분류기에서 나온 결과들 중 많은 수를 차지한 예측값을 그대로 사용한다. (횟수 중심, 다수결)

간접 투표는 여러 분류기에서 나온 예측들의 평균을 계산하여 가장 높은 확률을 가진 클래스를 예측값으로 사용한다.

### 배깅 앙상블의 훈련을 여러 대의 서버에 분산시켜 속도를 높일 수 있을까요? 페이스팅 앙상블, 부스팅 앙상블, 랜덤 포레스트, 스태킹 앙상블의 경우는 어떨까요?

배깅, 페이스팅, 랜덤 포레스트 앙상블의 예측기는 서로 독립이므로 여러 대의 서버에 분산시켜 훈련 시킬 경우 앙상블의 훈련 속도를 높일 수 있다.

그러나 부스팅은 이전의 모델을 보완해나가면서 만들어지기때문에 훈련이 순차적이어야하기에 분산 시킨다고 얻는 효과가 없다.

### oob 평가의 장점은 무엇일까요?

oob 평가는 각 예측기에 대한 어떤 샘플의 사용 유무를 평가할 때 사용된다. 선택되지 않은 샘플을 사용해서 평가하기에, 추가적인 검증 세트가 없어도 편향되지 않게 앙상블을 평가하도록 도와준다. 그러므로 더 많은 훈련 샘플을 사용하여 앙상블 모델의 성능을 향상시킬 수 있다.

### 무엇이 엑스트라 트리를 일반 랜덤 포레스트보다 더 무작위하게 만드나요? 추가적인 무작위성이 어떻게 도움이 될까요? 엑스트라 트리는 일반 랜덤 포레스트보다 느릴까요, 빠를까요?

### 에이다부스트 앙상블이 훈련 데이터에 과소적합되었다면 어떤 매개변수를 어떻게 바꾸어야 할까요?

학습기의 갯수(n_estimators)의 수를 증가시키거나, 규제 하이퍼파라미터를 감소시킨다. 학습률(learning_rate)를 증가시켜본다

### 그레디언트 부스팅 앙상블이 훈련 데이터에 과대적합되었다면 학습률을 높여야할까요, 낮춰야할까요?

학습률을 낮춰야한다

### MNIST 데이터를 불러들여 훈련 세트, 검증 세트, 테스트 세트로 나눈다(훈련  : 50000/ 검증 : 10000/ 테스트 : 10000개) 그런 다음 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기 같은 여러 종류의 분류기를 훈련시킨다. 그리고 검증 세트에서 개개의 분류기보다 더 높은 성능을 내도록 이들을 간접 또는 직접 투표 방법을 사용해 앙상블로 연결해보세요. 앙상블을 얻고나면 테스트 세트로 확인해보세요. 개개의 분류기와 비교해서 성능이 얼마나 향상되나요?

In [1]:
# mnist 불러오기
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1)

In [2]:
# test set 10000개 분리
from sklearn.model_selection import train_test_split
X_train_val,X_test,y_train_val,y_test = train_test_split(mnist['data'],mnist['target'],test_size = 10000, random_state = 42)

In [3]:
# val set 10000개 분리
X_train,X_val,y_train,y_val = train_test_split(X_train_val,y_train_val, test_size = 10000, random_state = 42)

In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 784)
(50000,)
(10000, 784)
(10000,)
(10000, 784)
(10000,)


In [5]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier 
from sklearn.neural_network import MLPClassifier

In [6]:
# 모델 설정
lsvm = SVC(max_iter = 1000, random_state = 42, probability = True, kernel = "linear") # 학습 반복횟수 1000번
rand = RandomForestClassifier(n_estimators = 10,random_state = 42) # 예측기의 개수 = 10개
extr = ExtraTreesClassifier(n_estimators = 10, random_state = 42)
mlp = MLPClassifier() # 다중신경망 분류

In [7]:
from sklearn.metrics import accuracy_score

In [8]:
model = [lsvm,rand,extr,mlp]

for i in model :
    print("훈련 :", i)
    i.fit(X_train,y_train)

훈련 : SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=1000, probability=True, random_state=42,
  shrinking=True, tol=0.001, verbose=False)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


훈련 : RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
훈련 : ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
훈련 : MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), l

In [ ]:
# val 검증
for i in model :
    y_pred = i.predict(X_val)
    print("정확도 :", accuracy_score(y_val,y_pred))

정확도 : 0.811
정확도 : 0.9469
정확도 : 0.9492
정확도 : 0.9638


성능은 mlp, extr, rand, svm 순

In [ ]:
# voting
from sklearn.ensemble import VotingClassifier

models = [("LinearSVM", lsvm),("ExtraTrees",extr),("RandomForest",rand),("MLPClassifier",mlp)]

In [ ]:
vot_clf = VotingClassifier(models)

In [ ]:
# vot 모델 적합
vot_clf.fit(X_train,y_train)

In [ ]:
# vto_clf 스코어
vot_clf.score(X_val,y_val)

In [ ]:
[model.score for model in vot_clf.estimators_]

In [ ]:
# 간접투표
# 이미 모델 훈련은 시켰기때문에 hard -> soft로 변환
vot_clf.voting = 'soft'

In [ ]:
vot_clf.score(X_val,y_val)

In [ ]:
# test 점수 예측
[model.score(X_test,y_test) for model in vot_clf.estimators_]